<a href="https://colab.research.google.com/github/MasahikoAtsumi/deep_lec/blob/main/obi_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**古代文字の認識**

ここで我々の研究室で長年引き継がれてきた、CNNを用いた古代文字の認識を試してみよう。


実行に移るまえに、データセット（学習の対象）を用意しなければいけない。

研究室のホームページから、obiデータセットをダウンロードし、それを(Imagesフォルダ)
ドライブにアップロードをしてほしい。

それが終わったら、順に下のコード実行しよう。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import keras

#keras.applications.xception.Xception(input_tensor=None, input_shape=None, pooling=None, classes=1000)
#difine model
model = keras.applications.xception.Xception(include_top=True,weights=None, input_tensor=None, input_shape=None, pooling=None, classes=29)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
#set dataset
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
        zoom_range=0.2,
    horizontal_flip=True,                            
    validation_split=0.2) 
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Images',
    target_size=(299,299),
    batch_size=8,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Images',
    target_size=(299,299),
    batch_size=8,
    class_mode='categorical',
    subset='validation')

#set the information about training
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 10,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 10,
    epochs = 10)

#save the model and the weight
json_string=model.to_json()
open('obi_cnn.json',"w").write(json_string)
model.save_weights('obi_cnn.h5')


import cv2
import os
for root, dirs, files in os.walk('/content/drive/My Drive/Images'):
        for i in files:
             if i.endswith(".pgm"):
                    os.remove(os.path.join(root, i))

少し時間がかかるかもしれない。

気長に待とう。

実行が終わったら、任意の画像を読み込んでみよう。



In [ ]:
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os

#load image
temp_img=load_img("/content/drive/My Drive/Images/ask/0000_67761.3_13_0.pgm.png",target_size=(299,299))

#normalize
temp_img_array=img_to_array(temp_img)
temp_img_array=temp_img_array.astype('float32')/255.0
temp_img_array=temp_img_array.reshape((1,299,299,3))

#load model
json_string=open('/content/obi_cnn.json').read()
model=model_from_json(json_string)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.load_weights('obi_cnn.h5')

#define class
path = '/content/drive/My Drive/Images/'
files = os.listdir(path)
files_dir = [f for f in files if os.path.isdir(os.path.join(path, f))]
files_dir = ['ajia', 'ame', 'ask', 'bou', 'dollar', 'five', 'hi', 'high', 'hinoe', 
             'hu', 'jin', 'kou', 'mi', 'ne', 'now', 'onore', 'otsu', 'paddy', 'rapit',
             'rou', 'sada', 'san', 'spics', 'stand', 'that', 'to', 'ushi', 'yoku', 'you']
  

#predict
img_pred=model.predict(temp_img_array)
print('\npredict_classes=',files_dir[np.argmax(img_pred)] )

plt.imshow(temp_img)
plt.title('pred:{}'.format(img_pred))
plt.show()

どうだろう？

正しく認識されただろうか？

選ぶクラスによっては正しく認識されない場合もある。

その場合はなぜ認識されなかったのかを考えてみよう。

本来であれば、学習に使われたデータを評価に用いるのはよくない。

研究の際は、学習とは別にテストデータを用意して評価しよう。